# Probabilistic Forecasting with `sktime`

### Overview of this notebook

* quick start - probabilistic forecasting
* disambiguation - types of probabilistic forecasts
* details: probabilistic forecasting interfaces
* metrics for, and evaluation of probabilistic forecasts
* advanced composition: pipelines, tuning, reduction
* extender guide
* contributor credits

In [ ]:
import warnings

warnings.filterwarnings("ignore")

---
### Quick Start - Probabilistic Forecasting with `sktime`

... works exactly like the basic forecasting workflow, replace `predict` by a probabilistic method!

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.theta import ThetaForecaster


# step 1: data specification
y = load_airline()
# step 2: specifying forecasting horizon
fh = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# step 3: specifying the forecasting algorithm
forecaster = ThetaForecaster(sp=12)
# step 4: fitting the forecaster
forecaster.fit(y, fh=fh)
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.9)
y_pred_int

for illustration, plotting actuals and prediction intervals:

In [ ]:
from sktime.utils import plotting

# also requires predictions
y_pred = forecaster.predict()

fig, ax = plotting.plot_series(
    y, y_pred, labels=["y", "y_pred"], pred_interval=y_pred_int
)

ax.legend();

**probabilistic forecasting methods in `sktime`**:

* forecast intervals    - `predict_interval(fh=None, X=None, coverage=0.90)`
* forecast quantiles    - `predict_quantiles(fh=None, X=None, alpha=[0.05, 0.95])`
* forecast variance     - `predict_var(fh=None, X=None, cov=False)`
* distribution forecast - `predict_proba(fh=None, X=None, marginal=True)`

To check which forecasters in `sktime` support probabilistic forecasting, use the `registry.all_estimators` utility and search for estimators which have the `capability:pred_int` tag (value `True`).

For composites such as pipelines, a positive tag means that logic is implemented if (some or all) components support it.

---
### Probabilistic forecasting interfaces in `sktime` 

This section:

* walkthrough of probabilistic predict methods
* multivariate and hierarchical data

All forecasters with tag `capability:pred_int` provide the following:

* forecast intervals    - `predict_interval(fh=None, X=None, coverage=0.90)`
* forecast quantiles    - `predict_quantiles(fh=None, X=None, alpha=[0.05, 0.95])`
* forecast variance     - `predict_var(fh=None, X=None, cov=False)`
* distribution forecast - `predict_proba(fh=None, X=None, marginal=True)`

Generalities:

* methods do not change state, multiple can be called
* `fh` is optional, if passed late
* exogeneous data `X` can be passed

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.theta import ThetaForecaster

# until fit, identical with the simple workflow
y = load_airline()

fh = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

forecaster = ThetaForecaster(sp=12)
forecaster.fit(y, fh=fh)

##### `predict_interval` - interval predictions

Inputs:\
`fh` - forecasting horizon (not necessary if seen in `fit`)\
`coverage`, float or list of floats, default=`0.9`\
nominal coverage(s) of the prediction interval(s) queried

Output: `pandas.DataFrame`\
Row index is `fh`\
Column has multi-index:\
1st level = variable name from y in fit\
2nd level = coverage fractions in `coverage`\
3rd level = string "lower" or "upper"\

Entries = forecasts of lower/upper interval at nominal coverage in 2nd lvl, for var in 1st lvl, for time in row

In [ ]:
coverage = 0.9
y_pred_ints = forecaster.predict_interval(coverage=coverage)
y_pred_ints

pretty-plotting the predictive interval forecasts:

In [ ]:
from sktime.utils import plotting

# also requires predictions
y_pred = forecaster.predict()

fig, ax = plotting.plot_series(
    y, y_pred, labels=["y", "y_pred"], pred_interval=y_pred_ints
)

ax.legend();

multiple coverages:

In [ ]:
coverage = [0.5, 0.9, 0.95]
y_pred_ints = forecaster.predict_interval(coverage=coverage)
y_pred_ints

##### `predict_quantiles` - quantile forecasts

Inputs:\
`fh` - forecasting horizon (not necessary if seen in `fit`)\
`alpha`, float or list of floats, default = `[0.1, 0.9]`\
quantile points at which quantiles are queried

Output: `pandas.DataFrame`\
Row index is `fh`\
Column has multi-index:\
1st level = variable name from y in fit\
2nd level = quantile points in `alpha`\

Entries = forecasts of quantiles at quantile point in 2nd lvl, for var in 1st lvl, for time in row

In [ ]:
alpha = [0.1, 0.25, 0.5, 0.75, 0.9]
y_pred_quantiles = forecaster.predict_quantiles(alpha=alpha)
y_pred_quantiles

pretty-plotting the quantile interval forecasts:

In [ ]:
from sktime.utils import plotting

columns = [y_pred_quantiles[i] for i in y_pred_quantiles.columns]
fig, ax = plotting.plot_series(y[-50:], *columns)

##### `predict_var` - variance forecasts

Inputs:\
`fh` - forecasting horizon (not necessary if seen in `fit`)\
`cov`, boolean, default=False\
whether covariance forecasts should also be returned (not all estimators support this)

Output: `pandas.DataFrame`, for cov=False:\
Row index is `fh`\
Column is equal to column index of `y` (variables)

Entries = variance forecast for variable in col, for time in row

In [ ]:
y_pred_variance = forecaster.predict_var()
y_pred_variance

##### `predict_proba` - distribution forecasts aka "full" probabilistic forecasts

Inputs:\
`fh` - forecasting horizon (not necessary if seen in `fit`)\

Output: `skpro` `Distribution` object

In [ ]:
y_pred_dist = forecaster.predict_proba()
y_pred_dist

In [ ]:
# obtaining quantiles
y_pred_dist.quantile([0.1, 0.9])

##### a note on consistence of methods

Outputs of `predict_interval`, `predict_quantiles`, `predict_var`, `predict_proba` are *typically* but not *necessarily* consistent with each other!

Consistency is weak interface requirement but not strictly enforced.

#### Probabilistic forecasting for multivariate and hierarchical data

multivariate data: first column index for different variables

In [ ]:
from sktime.datasets import load_longley
from sktime.forecasting.var import VAR

_, y = load_longley()

mv_forecaster = VAR()

mv_forecaster.fit(y, fh=[1, 2, 3])
# mv_forecaster.predict_var()

hierarchical data: probabilistic forecasts per level are row-concatenated with a row hierarchy index

In [ ]:
from sktime.forecasting.arima import ARIMA
from sktime.utils._testing.hierarchical import _make_hierarchical

y_hier = _make_hierarchical()
y_hier

In [ ]:
forecaster = ARIMA()
forecaster.fit(y_hier, fh=[1, 2, 3])
forecaster.predict_interval()

---
### Metrics for probabilistic forecasts and evaluation


Same as for `skpro`, see there!

* `predict_proba_function` outputs are the same as for `skpro`
* metrics in `sktime` are identical with metrics in `skpro`
* difference is how predictions are produced - forecasting, not regression

In [ ]:
import numpy as np

from sktime.datasets import load_airline
from sktime.forecasting.theta import ThetaForecaster

y_train = load_airline()[0:24]  # train on 24 months, 1949 and 1950
y_test = load_airline()[24:36]  # ground truth for 12 months in 1951

# try to forecast 12 months ahead, from y_train
fh = np.arange(1, 13)

forecaster = ThetaForecaster(sp=12)
forecaster.fit(y_train, fh=fh)

pred_quantiles = forecaster.predict_quantiles(alpha=[0.1, 0.25, 0.5, 0.75, 0.9])
pred_quantiles

In [ ]:
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss

pinball_loss = PinballLoss()
pinball_loss(y_true=y_test, y_pred=pred_quantiles)

In [ ]:
pinball_loss.evaluate_by_index(y_true=y_test, y_pred=pred_quantiles)

#### evaluation by backtesting

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss

# 1. define data
y = load_airline()

# 2. define splitting/backtesting regime
fh = [1, 2, 3]
cv = ExpandingWindowSplitter(step_length=12, fh=fh, initial_window=72)

# 3. define loss to use
loss = PinballLoss()
# default is score_average=True and multi_output="uniform_average", so gives a number

forecaster = ThetaForecaster(sp=12)
results = evaluate(
    forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=True, scoring=loss
)
results.iloc[:, :5].head()

* each row is one train/test split in the walkforward setting
* first col is the loss on the test fold
* last two columns summarize length of training window, cutoff between train/test

---
### Advanced composition: pipelines, tuning, reduction, adding proba forecasts to any estimator


composition = constructing "composite" estimators out of multiple "component" estimators

* **reduction** = building estimator type A using estimator type B
    * special case: adding proba forecasting capability to non-proba forecaster
    * special case: using proba supervised learner for  proba forecasting
* **pipelining** = chaining estimators, here: transformers to a forecaster
* **tuning** = automated hyper-parameter fitting, usually via internal evaluation loop
    * special case: grid parameter search and random parameter search tuning
    * special case: "Auto-ML", optimizing not just estimator hyper-parameter but also choice of estimator

#### Adding probabilistic forecasts to non-probabilistic forecasters

start with a forecaster that does not produce probabilistic predictions:

In [ ]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

my_forecaster = ExponentialSmoothing()

# does the forecaster support probabilistic predictions?
my_forecaster.get_tag("capability:pred_int")

adding probabilistic predictions is possible via reduction wrappers:

In [ ]:
# NaiveVariance adds intervals & variance via collecting past residuals
from sktime.forecasting.naive import NaiveVariance

# create a composite forecaster like this:
my_forecaster_with_proba = NaiveVariance(my_forecaster)

# does it support probabilistic predictions now?
my_forecaster_with_proba.get_tag("capability:pred_int")

the composite can now be used like any probabilistic forecaster:

In [ ]:
y = load_airline()

my_forecaster_with_proba.fit(y, fh=[1, 2, 3])
my_forecaster_with_proba.predict_interval()

wrappers of this type:

* `NaiveVariance` (the "baseline" of adding proba to forecasts)
* `ConformalIntervals`
* `BaggingForecaster`, usable with
    * `STLBootstrapTransformer`
    * `MovingBlockBootstrapTransformer`
    * `STLBootstrapTransformer`

#### Tuning and AutoML 

tuning and autoML with probabilistic forecasters works exactly like with "ordinary" forecasters\
via `ForecastingGridSearchCV` or `ForecastingRandomSearchCV`

* change metric to tune to a probabilistic metric
* use a corresponding probabilistic metric or loss function

Internally, evaluation will be done using probabilistic metric, via backtesting evaluation.

**important**: to evaluate the tuned estimator, use it in `evaluate` or a separate benchmarking workflow.\
Using internal metric/loss values amounts to in-sample evaluation, which is over-optimistic.

In [ ]:
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
)
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss

# forecaster we want to tune
forecaster = ThetaForecaster()

# parameter grid to search over
param_grid = {"sp": [1, 6, 12]}

# evaluation/backtesting regime for *tuning*
fh = [1, 2, 3]  # fh for tuning regime, does not need to be same as in fit/predict!
cv = SlidingWindowSplitter(window_length=36, fh=fh)
scoring = PinballLoss()

# construct the composite forecaster with grid search compositor
gscv = ForecastingGridSearchCV(
    forecaster, cv=cv, param_grid=param_grid, scoring=scoring, strategy="refit"
)

In [ ]:
from sktime.datasets import load_airline

y = load_airline()[:60]

gscv.fit(y, fh=fh)

inspect hyper-parameter fit obtained by tuning:

In [ ]:
gscv.best_params_

obtain predictions:

In [ ]:
gscv.predict_interval()

for AutoML, use the `MultiplexForecaster` to select among multiple forecasters:

In [ ]:
from sktime.forecasting.compose import MultiplexForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster, NaiveVariance

forecaster = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster(strategy="last")),
        ("ets", ExponentialSmoothing(trend="add", sp=12)),
    ],
)

forecaster_param_grid = {"selected_forecaster": ["ets", "naive"]}
gscv = ForecastingGridSearchCV(forecaster, cv=cv, param_grid=forecaster_param_grid)

gscv.fit(y)
gscv.best_params_

#### Pipelines with probabilistic forecasters

`sktime` pipelines are compatible with probabilistic forecasters:

* `ForecastingPipeline` applies transformers to the exogeneous `X` argument before passing them to the forecaster
* `TransformedTargetForecaster` transforms `y` and back-transforms forecasts, including interval or quantile forecasts

`ForecastingPipeline` takes a chain of transformers and forecasters, say,

`[t1, t2, ..., tn, f]`,

where `t[i]` are forecasters that pre-process, and `tp[i]` are forecasters that postprocess

##### `fit(y, X, fh)` does:

`X1 = t1.fit_transform(X)`\
`X2 = t2.fit_transform(X1)`\
etc\
`X[n] = t3.fit_transform(X[n-1])`\

`f.fit(y=y, x=X[n])`

##### `predict_[sth](X, fh)` does:

`X1 = t1.transform(X)`\
`X2 = t2.transform(X1)`\
etc\
`X[n] = t3.transform(X[n-1])`

`f.predict_[sth](X=X[n], fh)`

In [ ]:
from sktime.datasets import load_macroeconomic
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import ForecastingPipeline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.series.impute import Imputer

In [ ]:
data = load_macroeconomic()
y = data["unemp"]
X = data.drop(columns=["unemp"])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

In [ ]:
forecaster = ForecastingPipeline(
    steps=[
        ("imputer", Imputer(method="mean")),
        ("forecaster", ARIMA(suppress_warnings=True)),
    ]
)
forecaster.fit(y=y_train, X=X_train, fh=X_test.index[:5])
forecaster.predict_interval(X=X_test[:5])

`TransformedTargetForecaster` takes a chain of transformers and forecasters, say,

`[t1, t2, ..., tn, f, tp1, tp2, ..., tk]`,

where `t[i]` are forecasters that pre-process, and `tp[i]` are forecasters that postprocess

##### `fit(y, X, fh)` does:\
`y1 = t1.fit_transform(y)`\
`y2 = t2.fit_transform(y1)`\
`y3 = t3.fit_transform(y2)`\
etc\
`y[n] = t3.fit_transform(y[n-1])`

`f.fit(y[n])`

`yp1 = tp1.fit_transform(yn)`\
`yp2 = tp2.fit_transform(yp1)`\
`yp3 = tp3.fit_transform(yp2)`\
etc

##### `predict_quantiles(y, X, fh)` does:

`y1 = t1.transform(y)`\
`y2 = t2.transform(y1)`\
etc\
`y[n] = t3.transform(y[n-1])`

`y_pred = f.predict_quantiles(y[n])`

`y_pred = t[n].inverse_transform(y_pred)`\
`y_pred = t[n-1].inverse_transform(y_pred)`\
etc\
`y_pred = t1.inverse_transform(y_pred)`\
`y_pred = tp1.transform(y_pred)`\
`y_pred = tp2.transform(y_pred)`\
etc\
`y_pred = tp[n].transform(y_pred)`\

**Note**: the remaining proba predictions are inferred from `predict_quantiles`.

In [ ]:
from sktime.datasets import load_macroeconomic
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.detrend import Deseasonalizer, Detrender

In [ ]:
data = load_macroeconomic()
y = data[["unemp"]]

In [ ]:
forecaster = TransformedTargetForecaster(
    [
        ("deseasonalize", Deseasonalizer(sp=12)),
        ("detrend", Detrender()),
        ("forecast", ARIMA()),
    ]
)

forecaster.fit(y, fh=[1, 2, 3])
forecaster.predict_interval()

In [ ]:
forecaster.predict_quantiles()

quick creation also possible via the `*` dunder method, same pipeline:

In [ ]:
forecaster = Deseasonalizer(sp=12) * Detrender() * ARIMA()

In [ ]:
forecaster.fit(y, fh=[1, 2, 3])
forecaster.predict_interval()

---
## Building your own probabilistic forecaster

Getting started:

* follow the ["implementing estimator" developer guide](https://www.sktime.net/en/stable/developer_guide/add_estimators.html)
* use the advanced [forecasting extension template](https://github.com/sktime/sktime/blob/main/extension_templates/forecasting.py)

Extension template = python "fill-in" template with to-do blocks that allow you to implement your own, sktime-compatible forecasting algorithm.

Check estimators using `check_estimator`

For probabilistic forecasting:

* implement at least one of `predict_quantiles`, `predict_interval`, `predict_var`, `predict_proba`
* optimally, implement all, unless identical with defaulting behaviour as below
* if only one is implemented, others use following defaults (in this sequence, dependent availability):
    * `predict_interval` uses quantiles from `predict_quantiles` and vice versa
    * `predict_var` uses variance from `predict_proba`, or variance of normal with IQR as obtained from `predict_quantiles`
    * `predict_interval` or `predict_quantiles` uses quantiles from `predict_proba` distribution
    * `predict_proba` returns normal with mean `predict` and variance `predict_var`
* so if predictive residuals not normal, implement `predict_proba` or `predict_quantiles`
* if interfacing, implement the ones where least "conversion" is necessary
* ensure to set the `capability:pred_int` tag to `True`


In [ ]:
# estimator checking on the fly using check_estimator

# suppose NaiveForecaster is your new estimator
from sktime.forecasting.naive import NaiveForecaster

# check the estimator like this

# uncomment this block to run

# from sktime.utils.estimator_checks import check_estimator
#
# check_estimator(NaiveForecaster)

# this prints any failed tests, and returns dictionary with
#   keys of test runs and results from the test run
# run individual tests using the tests_to_run arg or the fixtures_to_run_arg
#   these need to be identical to test or test/fixture names, see docstring

In [ ]:
# to raise errors for use in traceback debugging:

# uncomment next line to run
# check_estimator(NaiveForecaster, raise_exceptions=True)

# this does not raise an error since NaiveForecaster is fine, but would if it weren't

---
## Summary

* unified API for probabilistic forecasting and probabilistic metrics
* integrating other packages (e.g. scikit-learn, statsmodels, pmdarima, prophet)
* interface for composite model building is same, proba or not (pipelining, ensembling, tuning, reduction)
* easily extensible with custom estimators

### Useful resources
* For more details, take a look at [our paper on forecasting with sktime](https://arxiv.org/abs/2005.08067) in which we discuss the forecasting API in more detail and use it to replicate and extend the M4 study.
* For a good introduction to forecasting, see [Hyndman, Rob J., and George Athanasopoulos. Forecasting: principles and practice. OTexts, 2018](https://otexts.com/fpp2/).
* For comparative benchmarking studies/forecasting competitions, see the [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128) and the [M5 competition](https://www.kaggle.com/c/m5-forecasting-accuracy/overview).


### Credits

notebook creation: fkiraly

probablistic forecasting framework: fkiraly, kejsitake\
probabilistic metrics, tuning: eenticott-shell, fkiraly\
probabilistic estimators: aiwalter, fkiraly, ilyasmoutawwakil, k1m190r, kejsitake